In [1]:
import os
import sys
import json
sys.path.append(os.path.join(os.getcwd().replace("model_inference", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.profiler import profiler
import matplotlib.pyplot as plt
import time
from parse_dataset import NetworkDataset, parse_dataset, split_datasets, binary_dataset
from model import Model
from load_models import models
from benchmark import Benchmark

In [ ]:
conf = {
    "batch_size": 128,
    "epochs": 10,
    "learning_rate": 0.0001,
    "model": "mlp",
    "device": "host"
}

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
dataset_path = os.path.join(os.getcwd().replace("model_inference", ""), "datasets")

# load label_dict
json_file = os.path.join(dataset_path, "label_index.json")
with open(json_file, 'r') as file:
    label_dict = json.load(file)

# load train, val and test datasets
train_dataset_file = os.path.join(dataset_path, "train_dataset.pt")
X_train, y_train = torch.load(train_dataset_file)

val_dataset_file = os.path.join(dataset_path, "val_dataset.pt")
X_val, y_val = torch.load(val_dataset_file)

test_dataset_file = os.path.join(dataset_path, "test_dataset.pt")
X_test, y_test = torch.load(test_dataset_file)

if conf["model"] != "mlp" and conf["model"] != "light_mlp":
    X_train, X_val, X_test = X_train.unsqueeze(-1), X_val.unsqueeze(-1), X_test.unsqueeze(-1)
    
print(X_train.shape)
print(y_train.shape)

# create train, val and test datasets
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=conf["batch_size"], shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=conf["batch_size"], shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=conf["batch_size"])

torch.Size([356334, 513])
torch.Size([356334])


In [5]:
model_path = os.path.join(os.getcwd().replace("model_inference", ""), "checkpoint", conf["model"] + "_model.pth")
gru_model = models[conf["model"]]
benchmark = Benchmark(gru_model, model_path, test_loader, conf["batch_size"], conf["model"])

Checkpoint loaded from /home/jorgetf/testmodel/Network-Packet-ML-Model/checkpoint/light_mlp_model.pth!


In [6]:
benchmark.memory_usage()
benchmark.latency()
benchmark.throughput()
benchmark.cpu_usage()
benchmark.accuracy()

[W1115 17:24:22.839047472 CPUAllocator.cpp:245] Memory block of unknown size was allocated before the profiling started, profiler results will not include the deallocation event
ERROR:2025-11-15 17:24:22 813931:813931 DeviceProperties.cpp:47] gpuGetDeviceCount failed with code 35


In [7]:
file = os.path.join(os.getcwd(), "benhcmarks", conf["device"], conf["model"] + "_" + conf["device"] + "_benchmark.txt")
with open(file, "w") as f:
    for line in benchmark.results:
        f.writelines(line + "\n")

for line in benchmark.results:
    print(line)

Benchmark - light_mlp model:

Memory usage (MB):
Avg memory usage: 0.227MB
Peak memory usage: 2.515MB

Model inference latency on one batch (batch size = 128):
Avg latency: 0.172ms
Min latency: 0.158ms
Max latency: 0.940ms

Model inference throughput (batch size = 128):
Throughput: 106536.66 samples/sec

CPU usage per thread:
Avg CPU usage: 25.7%
Max CPU usage: 89.9%
Number of threads used: 186.0

Model (light_mlp) inference accuracy (%):
Accuracy: 70.31%



